In [ ]:
pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 18.5 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
import csv

# Создание графа
g = Graph()

# Определение пространств имен
EX = Namespace("http://example.org/magical_creatures#")
DCT = Namespace("http://purl.org/dc/terms/")
SCHEMA = Namespace("https://schema.org/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")

g.bind("ex", EX)
g.bind("dct", DCT)
g.bind("schema", SCHEMA)
g.bind("foaf", FOAF)

def add_image(img_id, url):
    img_uri = EX[f"image_{img_id}"]
    g.add((img_uri, RDF.type, SCHEMA.ImageObject))
    g.add((img_uri, DCT.identifier, Literal(img_id)))
    g.add((img_uri, SCHEMA.contentUrl, Literal(url)))
    return img_uri

def add_audio(audio_id, url, duration, tone):
    audio_uri = EX[f"audio_{audio_id}"]
    g.add((audio_uri, RDF.type, SCHEMA.AudioObject))
    g.add((audio_uri, DCT.identifier, Literal(audio_id)))
    g.add((audio_uri, SCHEMA.contentUrl, Literal(url)))
    g.add((audio_uri, SCHEMA.duration, Literal(duration, datatype=XSD.float)))
    g.add((audio_uri, EX.tone, Literal(tone)))
    return audio_uri

def add_creature(row):
    creature_uri = EX[f"creature_{row['id']}"]

    # Основные свойства
    g.add((creature_uri, RDF.type, EX.MagicalCreature))
    g.add((creature_uri, EX.id, Literal(row["id"], datatype=XSD.integer)))
    g.add((creature_uri, EX.name, Literal(row["name"])))
    g.add((creature_uri, EX.universe, Literal(row["universe"].strip())))
    g.add((creature_uri, EX.type, Literal(row["type"])))
    g.add((creature_uri, EX.sex, Literal(row["sex"])))
    g.add((creature_uri, EX.habitat, Literal(row["habitat"])))

    # Физические характеристики
    g.add((creature_uri, EX.body_parts, Literal(row["body_parts"])))
    g.add((creature_uri, EX.color, Literal(row["color"])))
    g.add((creature_uri, EX.size, Literal(row["size"])))

    # Медиаресурсы
    img_id = row["picture_id "].strip()
    audio_id = row["audio_id"]

    img_uri = EX[f"image_{img_id}"]
    audio_uri = EX[f"audio_{audio_id}"]

    g.add((creature_uri, EX.hasImage, img_uri))
    g.add((creature_uri, EX.hasAudio, audio_uri))

    # Добавление медиаресурсов
    add_image(img_id, row["picture"])
    add_audio(
        audio_id,
        row["audio"],
        float(row["audio_duration"]) if row["audio_duration"] else 0.0,
        row["audio_tone"]
    )

    return img_uri, audio_uri

# Загрузка данных из CSV
with open("Magical creatures - Лист1 (1).csv", encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        try:
            add_creature(row)
        except Exception as e:
            print(f"Ошибка обработки строки {row}: {str(e)}")

# Сохранение графа
g.serialize("magical_creatures_full.ttl", format="turtle")
print(f"Граф сохранен, содержит {len(g)} триплетов")

Граф сохранен, содержит 40 триплетов


In [ ]:
import pandas as pd
query1 = prepareQuery('''
    SELECT ?id ?name ?universe ?imageUrl ?audioUrl
    WHERE {
        ?creature a ex:MagicalCreature ;
                  ex:id ?id ;
                  ex:name ?name ;
                  ex:universe ?universe ;
                  ex:hasImage ?image ;
                  ex:hasAudio ?audio .
        ?image schema:contentUrl ?imageUrl .
        ?audio schema:contentUrl ?audioUrl .
    }
    ORDER BY ?id
''', initNs={"ex": EX, "schema": SCHEMA})

print("Все существа с медиаресурсами:")
results = []
for row in g.query(query1):
    results.append({
        "ID": row.id.value,
        "Name": row.name.value,
        "Universe": row.universe.value,
        "Image": row.imageUrl.value,
        "Audio": row.audioUrl.value
    })
pd.DataFrame(results)

Все существа с медиаресурсами:


,ID,Name,Universe,Image,Audio
0,1,murloc_tinyfin,world_of_warcraft,https://static.wikia.nocookie.net/hearthstone_...,https://hearthstonesounds.s3.amazonaws.com/LOE...
1,2,vaporeon,pokemon,https://mktg-assets.tcgplayer.com/fit-in/1000x...,https://us-tuna-sounds-files.voicemod.net/5c66...


In [ ]:
query2 = prepareQuery('''
    SELECT ?name ?habitat ?size ?color
    WHERE {
        ?creature a ex:MagicalCreature ;
                  ex:name ?name ;
                  ex:habitat ?habitat ;
                  ex:size ?size ;
                  ex:color ?color .
        FILTER(contains(LCASE(?habitat), "water"))
    }
''', initNs={"ex": EX})

print("\nВодные существа:")
results = []
for row in g.query(query2):
    results.append({
        "Name": row.name.value,
        "Habitat": row.habitat.value,
        "Size": row.size.value,
        "Color": row.color.value
    })
pd.DataFrame(results)


Водные существа:


,Name,Habitat,Size,Color
0,murloc_tinyfin,"water, swamp",medium,"blue, orange, green, red"
1,vaporeon,"water, urban areas",medium,"blue, white"


In [ ]:
query3 = prepareQuery('''
    SELECT ?audioUrl ?duration ?tone
    WHERE {
        ?audio a schema:AudioObject ;
               schema:contentUrl ?audioUrl ;
               schema:duration ?duration ;
               ex:tone ?tone .
        FILTER(?duration > 1.0)
    }
    ORDER BY DESC(?duration)
''', initNs={"schema": SCHEMA, "ex": EX})

print("\nДлинные аудиофайлы(> 1 cек.):")
results = []
for row in g.query(query5):
    results.append({
        "Audio URL": row.audioUrl.value,
        "Duration": f"{row.duration.value} sec",
        "Tone": row.tone.value
    })
pd.DataFrame(results)


Длинные аудиофайлы(> 1 cек.):


,Audio URL,Duration,Tone
0,https://us-tuna-sounds-files.voicemod.net/5c66...,2.0 sec,soft


In [ ]:
query4 = prepareQuery('''
    SELECT ?name ?bodyParts
    WHERE {
        ?creature a ex:MagicalCreature ;
                  ex:name ?name ;
                  ex:body_parts ?bodyParts .
    }
''', initNs={"ex": EX})

print("\nСущества с их частями тела:")
for row in g.query(query6):
    print(f"{row.name.value}: {row.bodyParts.value}")



Существа с их частями тела:
murloc_tinyfin: gills, tail
vaporeon: fins, tail, gills


In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
import csv

# Создание графа
g = Graph()

# Определение пространств имен
EX = Namespace("http://example.org/magical_creatures#")
DCT = Namespace("http://purl.org/dc/terms/")
SCHEMA = Namespace("https://schema.org/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")

g.bind("ex", EX)
g.bind("dct", DCT)
g.bind("schema", SCHEMA)
g.bind("foaf", FOAF)

def add_image(img_id, url):
    img_uri = EX[f"image_{img_id}"]
    g.add((img_uri, RDF.type, SCHEMA.ImageObject))
    g.add((img_uri, DCT.identifier, Literal(img_id)))
    g.add((img_uri, SCHEMA.contentUrl, Literal(url)))
    return img_uri

def add_audio(audio_id, url, duration, tone):
    audio_uri = EX[f"audio_{audio_id}"]
    g.add((audio_uri, RDF.type, SCHEMA.AudioObject))
    g.add((audio_uri, DCT.identifier, Literal(audio_id)))
    g.add((audio_uri, SCHEMA.contentUrl, Literal(url)))
    g.add((audio_uri, SCHEMA.duration, Literal(duration, datatype=XSD.float)))
    g.add((audio_uri, EX.tone, Literal(tone)))
    return audio_uri

def add_creature(row):
    creature_uri = EX[f"creature_{row['id']}"]

    # Основные свойства
    g.add((creature_uri, RDF.type, EX.MagicalCreature))
    g.add((creature_uri, EX.id, Literal(row["id"], datatype=XSD.integer)))
    g.add((creature_uri, EX.name, Literal(row["name"])))
    g.add((creature_uri, EX.universe, Literal(row["universe"].strip())))
    g.add((creature_uri, EX.type, Literal(row["type"])))
    g.add((creature_uri, EX.sex, Literal(row["sex"])))
    g.add((creature_uri, EX.habitat, Literal(row["habitat"])))

    # Физические характеристики
    g.add((creature_uri, EX.body_parts, Literal(row["body_parts"])))
    g.add((creature_uri, EX.color, Literal(row["color"])))
    g.add((creature_uri, EX.size, Literal(row["size"])))

    # Медиаресурсы
    img_id = row["picture_id"].strip()
    audio_id = row["audio_id"]

    img_uri = EX[f"image_{img_id}"]
    audio_uri = EX[f"audio_{audio_id}"]

    g.add((creature_uri, EX.hasImage, img_uri))
    g.add((creature_uri, EX.hasAudio, audio_uri))

    # Добавление медиаресурсов
    add_image(img_id, row["picture"])
    add_audio(
        audio_id,
        row["audio"],
        float(row["audio_duration"]) if row["audio_duration"] else 0.0,
        row["audio_tone"]
    )

    return img_uri, audio_uri

# Список CSV-файлов для загрузки
csv_files = [
    "Magical creatures - Лист1 (1).csv",
    "Magical creatures - audio.csv",
    "Magical creatures - picture.csv"
]

# Загрузка данных из всех CSV-файлов
for csv_file in csv_files:
    try:
        with open(csv_file, encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                try:
                    add_creature(row)
                except Exception as e:
                    print(f"Ошибка обработки строки {row} в файле {csv_file}: {str(e)}")
    except FileNotFoundError:
        print(f"Файл {csv_file} не найден и будет пропущен.")
    except Exception as e:
        print(f"Ошибка при обработке файла {csv_file}: {str(e)}")

# Сохранение графа
g.serialize("magical_creatures_full.ttl", format="turtle")
print(f"Граф сохранен, содержит {len(g)} триплетов")

Ошибка обработки строки {'id': '1', 'picture_id ': '187605801', 'audio_id': '187606624', 'name': 'murloc_tinyfin', 'universe': 'world_of_warcraft', 'type': 'murlock', 'sex': 'male', 'habitat': 'water, swamp', 'picture': 'https://static.wikia.nocookie.net/hearthstone_gamepedia/images/f/f5/Murloc_Tinyfin_full.jpg/revision/latest/scale-to-width-down/1000?cb=20151107225208', 'body_parts': 'gills, tail', 'color': 'blue, orange, green, red ', 'size': 'medium', 'audio': 'https://hearthstonesounds.s3.amazonaws.com/LOEA10_3_BabyMurloc_Play.wav', 'audio_duration': '1', 'audio_tone': 'medium'} в файле Magical creatures - Лист1 (1).csv: 'picture_id'
Ошибка обработки строки {'id': '2', 'picture_id ': '187608084', 'audio_id': '187608532', 'name': 'vaporeon', 'universe': 'pokemon', 'type': 'pokemon', 'sex': 'female', 'habitat': 'water, urban areas', 'picture': 'https://mktg-assets.tcgplayer.com/fit-in/1000x1000/filters:quality(75)/content/pokemon/9_21/13/C-08-16-2021-PKM-OG-Vaporeon@2x.jpg', 'body_pa